In [1]:
import numpy as np
import deepthought, mne, os

mne.set_log_level('WARNING')
# mne.set_log_level('INFO')
verbose = False

from openmiir.constants import STIMULUS_IDS
from openmiir.metadata import load_stimuli_metadata, get_stimuli_version
from openmiir.events import get_event_id
from deepthought.util.fs_util import save

In [2]:
sfreq = 512  # no down-sampling

# select data to be extracted
# subjects = ['P01','P04','P05','P06','P07','P09','P11','P12','P13','P14']
subjects = ['P01','P04','P06','P07','P09','P11','P12','P13','P14']  # excluding P05 #initial program
# subjects = ['P01','P04','P06','P07','P09']  # excluding P05 gaomaosen：All the files after 11 have errors

conditions = [1]
stimuli = STIMULUS_IDS

include_cue = False
use_mastoid_reference = False

data_root = 'E:\\OpenMIIR-RawEEG_v1\\OpenMIIR'
mne_data_root = os.path.join(data_root, 'eeg', 'mne')
ica_data_root = os.path.join(data_root, 'eeg', 'preprocessing', 'ica')
target_path = 'data/OpenMIIR-Perception-512Hz.pklz'

import warnings
warnings.filterwarnings("ignore")

In [3]:
cache = dict()

In [4]:
from openmiir.preprocessing import load_and_preprocess_raw
from openmiir.preprocessing.pipeline import load_raw_info

for subject in subjects:
    print subject
    if subject in cache: continue
    
    info = load_raw_info(subject, mne_data_root=mne_data_root)  # need to keep info with bad channels for later

    raw, trial_events = load_and_preprocess_raw(subject, 
                                            mne_data_root=mne_data_root,
                                            ica_data_root=ica_data_root,
                                            onsets='audio',
                                            interpolate_bad_channels=True, 
                                            reference_mastoids=use_mastoid_reference,
                                            l_freq=0.5,
                                            h_freq=30,
                                            sfreq=sfreq,
                                            ica_cleaning=True,
                                            verbose=None,
                                            n_jobs=4)
    cache[subject] = raw, trial_events, info

P01
The behavior of set_eeg_reference will change in 0.16 when ref_channels="average". Currently, a projection is computed, which has to be applied manually with the apply_proj method. In 0.16, the average reference will be directly applied. Set projection=True if you want to retain the old behavior, or set projection=False if you want the new behavior.
An average reference projection was already added. The data has been left untouched.
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
Attenuation at stop frequency 0.4Hz is only 11.6dB. Increase filter_length for higher attenuation.
P04
The behavior of set_eeg_reference will change in 0.16 when ref_channels="average". Currently, a projection is computed, which has to be applied manually with the apply_proj method. In 0.16, the average reference will be directly applied. Set projection=True if you want to retain the old behavior, or set projection=False if you want the

In [5]:
trials = []
metadata = []
for subject in subjects:    
    # load metadata
    meta_version = get_stimuli_version(subject)
    meta = load_stimuli_metadata(data_root, meta_version)
    print subject, meta_version, raw.info['bads']
    
    raw, trial_events, info = cache[subject]
    for stim_id in stimuli:
        if include_cue:
            trial_len = meta[stim_id]['length_with_cue']
        else:
            trial_len = meta[stim_id]['length_without_cue']

        for cond in conditions:
            event_id = get_event_id(stim_id, cond)
            
            # select EEG channels
            eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, exclude=[])

            epochs = mne.Epochs(raw, events=trial_events, event_id=event_id, 
                    tmin=0, tmax=trial_len, 
                    proj=False, picks=eeg_picks, preload=True, verbose=False)
            
            
            # TEMPORARY FIX for update from mne-python 0.9 to 0.10
            # interpolate bad channels again as ICA will not be applied on them
            epochs.info['bads'] = info['bads']  # set bad channel flags for interpolation
            epochs.interpolate_bads()

            for i, trial in enumerate(epochs.get_data()):                                

                trials.append(np.asarray(trial, dtype=np.float32))

                trial_meta = dict()
                trial_meta['subject'] = subject
                if cond == 1:
                    trial_meta['trial_type'] = 'perception'
                else:
                    trial_meta['trial_type'] = 'imagination'
                trial_meta['trial_no'] = i
                trial_meta['condition'] = cond
                trial_meta['stimulus_id'] = stim_id

#                 print trial_meta
                metadata.append(trial_meta)                

target_path = 'data/OpenMIIR-Perception-512Hz.pklz'
print 'saving dataset to', target_path
print len(trials), len(metadata)
save(target_path, (trials, metadata), mkdirs=True)

P01 1 []
P04 1 []
P06 1 []
P07 1 []
P09 2 []
P11 2 []
P12 2 []
P13 2 []
P14 2 []
saving dataset to data/OpenMIIR-Perception-512Hz.pklz
540 540


In [6]:
min_len, max_len = 10000, 0
for trial in trials:
    min_len = min(min_len, trial.shape[-1])
    max_len = max(max_len, trial.shape[-1])
print min_len, max_len

3519 8201


In [7]:
# The last program ran for a long time
print trials

[array([[ 0.0000000e+00,  5.6654574e-07,  9.4770229e-07, ...,
        -3.0842957e-06, -3.0507813e-06, -2.9941671e-06],
       [ 0.0000000e+00,  4.1471995e-07,  6.4184957e-07, ...,
        -1.3471758e-06, -1.5784943e-06, -1.5030877e-06],
       [ 0.0000000e+00,  7.2672566e-07,  1.0778587e-06, ...,
        -1.1700496e-05, -1.1884496e-05, -1.1972387e-05],
       ...,
       [ 0.0000000e+00,  4.7192617e-07,  9.6520250e-07, ...,
         7.4697105e-06,  6.0544326e-06,  4.7950130e-06],
       [ 0.0000000e+00,  4.4416819e-07,  9.0415398e-07, ...,
        -1.3944468e-06, -2.4108567e-06, -3.5124749e-06],
       [ 0.0000000e+00,  5.5440415e-07,  1.0685884e-06, ...,
         5.6433396e-06,  5.4779030e-06,  5.0844278e-06]], dtype=float32), array([[0.0000000e+00, 3.3194851e-07, 7.4985252e-07, ..., 1.1670952e-05,
        1.2599744e-05, 1.3585257e-05],
       [0.0000000e+00, 1.5551286e-07, 4.4984458e-07, ..., 8.8788584e-06,
        9.5720925e-06, 1.0197510e-05],
       [0.0000000e+00, 3.0748285e-07, 

In [8]:
print metadata

[{'trial_type': 'perception', 'trial_no': 0, 'stimulus_id': 1, 'condition': 1, 'subject': 'P01'}, {'trial_type': 'perception', 'trial_no': 1, 'stimulus_id': 1, 'condition': 1, 'subject': 'P01'}, {'trial_type': 'perception', 'trial_no': 2, 'stimulus_id': 1, 'condition': 1, 'subject': 'P01'}, {'trial_type': 'perception', 'trial_no': 3, 'stimulus_id': 1, 'condition': 1, 'subject': 'P01'}, {'trial_type': 'perception', 'trial_no': 4, 'stimulus_id': 1, 'condition': 1, 'subject': 'P01'}, {'trial_type': 'perception', 'trial_no': 0, 'stimulus_id': 2, 'condition': 1, 'subject': 'P01'}, {'trial_type': 'perception', 'trial_no': 1, 'stimulus_id': 2, 'condition': 1, 'subject': 'P01'}, {'trial_type': 'perception', 'trial_no': 2, 'stimulus_id': 2, 'condition': 1, 'subject': 'P01'}, {'trial_type': 'perception', 'trial_no': 3, 'stimulus_id': 2, 'condition': 1, 'subject': 'P01'}, {'trial_type': 'perception', 'trial_no': 4, 'stimulus_id': 2, 'condition': 1, 'subject': 'P01'}, {'trial_type': 'perception', 